In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

2025-12-24 10:54:59.941359: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-24 10:55:00.114808: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-24 10:55:01.413469: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/sagnik/miniforge3/envs/ml/lib/python3.13/site-packages/keras/src/export/tf

In [2]:
datafolder = 'data'
healthy_data = os.path.join(datafolder,'Apple___healthy')
disease_data = os.path.join(datafolder,'Apple___disease')

In [3]:
print(f'{os.path.exists(healthy_data)=}\n{os.path.exists(disease_data)=}')

os.path.exists(healthy_data)=True
os.path.exists(disease_data)=True


# Model A: Shallow CNN

In [4]:
model_A = models.Sequential([
    layers.Input(shape=(256, 256, 3)),
    
    layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.Conv2D(filters=36, kernel_size=(3, 3), activation='relu', padding='same'),
    
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    layers.Flatten(),
    
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

model_A.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_A.summary()

I0000 00:00:1766553903.131887    2383 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2141 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 36)   │         1,980 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 36)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 589824)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    75,497,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,508,134 (288.04 MB)

 Trainable params: 75,508,134 (288.04 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
BATCH_SIZE = 2
IMG_SIZE = (256, 256)

In [6]:
train_ds_A = tf.keras.utils.image_dataset_from_directory(
    datafolder,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds_A = tf.keras.utils.image_dataset_from_directory(
    datafolder,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 3171 files belonging to 2 classes.
Using 2537 files for training.
Found 3171 files belonging to 2 classes.
Using 634 files for validation.


In [7]:
normalization_layer = layers.Rescaling(1./255)

train_ds_A = train_ds_A.map(lambda x, y: (normalization_layer(x), y))
val_ds_A = val_ds_A.map(lambda x, y: (normalization_layer(x), y))

In [8]:
history = model_A.fit(
    train_ds_A,
    validation_data=val_ds_A,
    epochs=30
)

Epoch 1/30


2025-12-24 10:55:05.192811: I external/local_xla/xla/service/service.cc:163] XLA service 0x7f28ac018960 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-24 10:55:05.192850: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2025-12-24 10:55:05.215153: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-24 10:55:05.347766: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


   7/1269 ━━━━━━━━━━━━━━━━━━━━ 26s 21ms/step - accuracy: 0.6221 - loss: 9.8698

I0000 00:00:1766553908.024480    2448 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1269/1269 ━━━━━━━━━━━━━━━━━━━━ 34s 24ms/step - accuracy: 0.8376 - loss: 0.5034 - val_accuracy: 0.8817 - val_loss: 0.3182
Epoch 2/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 28s 22ms/step - accuracy: 0.8959 - loss: 0.2723 - val_accuracy: 0.8927 - val_loss: 0.2594
Epoch 3/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 28s 22ms/step - accuracy: 0.9393 - loss: 0.1770 - val_accuracy: 0.8817 - val_loss: 0.3549
Epoch 4/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 28s 22ms/step - accuracy: 0.9495 - loss: 0.1389 - val_accuracy: 0.8155 - val_loss: 0.5962
Epoch 5/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 28s 22ms/step - accuracy: 0.9527 - loss: 0.1426 - val_accuracy: 0.8580 - val_loss: 0.4974
Epoch 6/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 28s 22ms/step - accuracy: 0.9657 - loss: 0.1016 - val_accuracy: 0.8644 - val_loss: 0.4936
Epoch 7/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 28s 22ms/step - accuracy: 0.9799 - loss: 0.0687 - val_accuracy: 0.8265 - val_loss: 0.7101
Epoch 8/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 28s 22ms/step - accuracy: 0.9862 - loss: 0.04

In [9]:
history

In [10]:
model_A.evaluate(val_ds_A)

317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8927 - loss: 1.7650


[1.7650033235549927, 0.8927444815635681]

In [11]:
img = tf.keras.utils.load_img(
    "data/Apple___disease/2aa2ed70-b0b7-4d30-9c84-f6ded6a03fc2___JR_FrgE.S 8739.JPG",
    target_size=(256, 256)
)

img_array = tf.keras.utils.img_to_array(img)
img_array = img_array / 255.0
img_array = tf.expand_dims(img_array, axis=0)  # shape (1, 256, 256, 3)

pred = model_A.predict(img_array)

predicted_class = tf.argmax(pred, axis=1).numpy()[0]
confidence = tf.reduce_max(pred).numpy()

print("Predicted class:", predicted_class)
print("Confidence:", confidence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step
Predicted class: 0
Confidence: 0.9999999


# Model B: Deep CNN

In [12]:
model_B = models.Sequential([
    layers.Input(shape=(256, 256, 3)),
    
    # Block 1
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Block 2
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Block 3
    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    layers.Flatten(),
    
    layers.Dense(256, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_B.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_B.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │    33,554,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,719,873 (128.63 MB)

 Trainable params: 33,719,873 (128.63 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
train_ds_B = tf.keras.utils.image_dataset_from_directory(
    datafolder,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds_B = tf.keras.utils.image_dataset_from_directory(
    datafolder,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 3171 files belonging to 2 classes.
Using 2537 files for training.
Found 3171 files belonging to 2 classes.
Using 634 files for validation.


In [14]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
])

In [15]:
normalization_layer = layers.Rescaling(1./255)

train_ds_B = train_ds_B.map(
    lambda x, y: (normalization_layer(data_augmentation(x, training=True)), y)
)

val_ds_B = val_ds_B.map(
    lambda x, y: (normalization_layer(x), y)
)

In [16]:
'''datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,        # rotate images
    zoom_range=0.2,           # zoom in/out
    horizontal_flip=True,     # left-right flip
    vertical_flip=False,      # usually False unless orientation doesn't matter
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=0.2
)'''

"datagen = ImageDataGenerator(\n    rescale=1.0 / 255.0,\n    rotation_range=20,        # rotate images\n    zoom_range=0.2,           # zoom in/out\n    horizontal_flip=True,     # left-right flip\n    vertical_flip=False,      # usually False unless orientation doesn't matter\n    width_shift_range=0.1,\n    height_shift_range=0.1,\n    validation_split=0.2\n)"

In [17]:
'''train_generator = datagen.flow_from_directory(
    directory=datafolder,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='training',     # IMPORTANT
    shuffle=True,
    seed=42
)'''

"train_generator = datagen.flow_from_directory(\n    directory=datafolder,\n    target_size=(256, 256),\n    batch_size=32,\n    class_mode='binary',\n    subset='training',     # IMPORTANT\n    shuffle=True,\n    seed=42\n)"

In [18]:
'''val_generator = datagen.flow_from_directory(
    directory=datafolder,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation',   # IMPORTANT
    shuffle=False,
    seed=42
)'''

"val_generator = datagen.flow_from_directory(\n    directory=datafolder,\n    target_size=(256, 256),\n    batch_size=32,\n    class_mode='binary',\n    subset='validation',   # IMPORTANT\n    shuffle=False,\n    seed=42\n)"

In [19]:
'''print(train_generator.class_indices)'''

'print(train_generator.class_indices)'

In [20]:
history = model_B.fit(
    train_ds_B,
    validation_data=val_ds_B,
    epochs=30
)

Epoch 1/30


2025-12-24 11:09:16.087291: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.05GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-12-24 11:09:16.157945: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.09GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-12-24 11:09:16.203553: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.05GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-12-24 11:09:17.136038: W external/local_xla/xla/ts

1267/1269 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5054 - loss: 0.7018

2025-12-24 11:09:44.995394: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-12-24 11:09:45.052059: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-12-24 11:09:45.091296: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-12-24 11:09:45.748468: W external/local_xla/xla/ts

1269/1269 ━━━━━━━━━━━━━━━━━━━━ 36s 24ms/step - accuracy: 0.5140 - loss: 0.6943 - val_accuracy: 0.5237 - val_loss: 0.6921
Epoch 2/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 27s 21ms/step - accuracy: 0.5175 - loss: 0.6928 - val_accuracy: 0.5237 - val_loss: 0.6921
Epoch 3/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 27s 21ms/step - accuracy: 0.5175 - loss: 0.6928 - val_accuracy: 0.5237 - val_loss: 0.6920
Epoch 4/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 27s 21ms/step - accuracy: 0.5175 - loss: 0.6928 - val_accuracy: 0.5237 - val_loss: 0.6920
Epoch 5/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 27s 21ms/step - accuracy: 0.5175 - loss: 0.6928 - val_accuracy: 0.5237 - val_loss: 0.6920
Epoch 6/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 26s 21ms/step - accuracy: 0.5175 - loss: 0.6928 - val_accuracy: 0.5237 - val_loss: 0.6920
Epoch 7/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 26s 21ms/step - accuracy: 0.5175 - loss: 0.6928 - val_accuracy: 0.5237 - val_loss: 0.6920
Epoch 8/30
1269/1269 ━━━━━━━━━━━━━━━━━━━━ 27s 21ms/step - accuracy: 0.5175 - loss: 0.69

In [21]:
model_B.evaluate(val_ds_B)

317/317 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5237 - loss: 0.6920


[0.6920329928398132, 0.5236592888832092]

In [22]:
img = tf.keras.utils.load_img(
    "data/Apple___disease/2aa2ed70-b0b7-4d30-9c84-f6ded6a03fc2___JR_FrgE.S 8739.JPG",
    target_size=(256, 256)
)

img_array = tf.keras.utils.img_to_array(img)
img_array = img_array / 255.0
img_array = tf.expand_dims(img_array, axis=0)

pred = model_B.predict(img_array)[0][0]

predicted_class = int(pred > 0.5)
confidence = pred if predicted_class == 1 else 1 - pred

print("Predicted class:", predicted_class)
print("Confidence:", confidence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 644ms/step
Predicted class: 1
Confidence: 0.5217541


In [23]:
model_A.save("model_A.keras")
model_B.save("model_B.keras")